In [ ]:
import cv2
import numpy as np
import face_recognition
import pickle
import os
import time
import threading
import logging
from datetime import datetime
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from IPython.display import display, clear_output, HTML
import ipywidgets as widgets
from PIL import Image, ImageDraw, ImageFont
import pandas as pd
import psutil
import warnings
warnings.filterwarnings('ignore')

# Deep learning imports
import torch
import torch.nn as nn
import torchvision.transforms as transforms

# Custom modules - Fixed imports
from cuda_utils import (
    cuda_manager, 
    print_system_info, 
    benchmark_device, 
    get_optimal_device,
    run_performance_benchmark,  # This function exists in cuda_utils.py
    get_cuda_info,
    get_memory_info,
    optimize_memory
)
from face_database import FaceDatabase
from face_recognition_system import FaceRecognitionSystem
from download_models import download_models

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('face_recognition.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

print("✅ All imports loaded successfully!")
print(f"OpenCV version: {cv2.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / (1024**3):.1f} GB")

# Check what detectors are available
print("\n🔍 Available Face Detectors:")
try:
    face_system = FaceRecognitionSystem()
    for detector_name in face_system.detectors.keys():
        detector = face_system.detectors[detector_name]
        if hasattr(detector, 'mtcnn') and detector.mtcnn is None and detector_name == 'mtcnn':
            print(f"  ⚠️  {detector_name.upper()}: Not available (facenet_pytorch not installed)")
        elif hasattr(detector, 'net') and detector.net is None and detector_name == 'dnn':
            print(f"  ✅ {detector_name.upper()}: Models available, ready to test")
        else:
            print(f"  ✅ {detector_name.upper()}: Ready")
    
    # Display database status
    db_count = face_system.database.get_face_count()
    print(f"\n📁 Face Database: {db_count} faces loaded")
    
except Exception as e:
    print(f"  ❌ Error initializing detectors: {e}")

print("\n🎯 System is ready for face recognition!")
print("📋 Next steps:")
print("  1. Run Cell 2 for system info")
print("  2. Run Cell 3 for database management")
print("  3. Add faces using the webcam interface")
print("  4. Start real-time recognition!")

e:\Documents\Personal-Projects\Python\FaceReconigtion\.venv\Lib\site-packages\face_recognition_models\__init__.py:7: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


In [3]:
# Initialize face recognition system
face_system = FaceRecognitionSystem()

# Create interactive widgets for database management
def create_database_interface():
    # Widgets
    name_input = widgets.Text(
        value='',
        placeholder='Enter person name',
        description='Name:',
        style={'description_width': 'initial'}
    )
    
    description_input = widgets.Textarea(
        value='',
        placeholder='Optional description',
        description='Description:',
        style={'description_width': 'initial'}
    )
    
    add_button = widgets.Button(
        description='Add Face from Webcam',
        button_style='success',
        icon='plus'
    )
    
    remove_button = widgets.Button(
        description='Remove Face',
        button_style='danger',
        icon='minus'
    )
    
    list_button = widgets.Button(
        description='List All Faces',
        button_style='info',
        icon='list'
    )
    
    stats_button = widgets.Button(
        description='Show Statistics',
        button_style='warning',
        icon='bar-chart'
    )
    
    output = widgets.Output()
    
    # Event handlers
    def add_face(b):
        with output:
            if not name_input.value.strip():
                print("❌ Please enter a name")
                return
                
            print(f"📸 Capturing face for {name_input.value}...")
            
            # Capture from webcam
            cap = cv2.VideoCapture(0)
            if not cap.isOpened():
                print("❌ Could not open webcam")
                return
            
            print("Position your face in the camera and press SPACE to capture, ESC to cancel")
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Show preview
                cv2.imshow('Face Capture - Press SPACE to capture, ESC to cancel', frame)
                
                key = cv2.waitKey(1) & 0xFF
                if key == ord(' '):  # Space to capture
                    success = face_system.add_face_from_frame(
                        frame, 
                        name_input.value.strip(),
                        description_input.value.strip()
                    )
                    if success:
                        print(f"✅ Successfully added face for {name_input.value}")
                        name_input.value = ''
                        description_input.value = ''
                    else:
                        print("❌ Failed to add face - no face detected or encoding failed")
                    break
                elif key == 27:  # ESC to cancel
                    print("❌ Capture cancelled")
                    break
            
            cap.release()
            cv2.destroyAllWindows()
    
    def remove_face(b):
        with output:
            if not name_input.value.strip():
                print("❌ Please enter a name to remove")
                return
                
            success = face_system.remove_face(name_input.value.strip())
            if success:
                print(f"✅ Successfully removed face(s) for {name_input.value}")
                name_input.value = ''
            else:
                print(f"❌ No faces found for {name_input.value}")
    
    def list_faces(b):
        with output:
            faces = face_system.database.list_all_faces()
            if not faces:
                print("📭 No faces in database")
                return
                
            print("👥 Faces in Database:")
            print("=" * 50)
            for face in faces:
                print(f"Name: {face['name']}")
                print(f"ID: {face['face_id']}")
                print(f"Added: {face['added_date']}")
                if face.get('description'):
                    print(f"Description: {face['description']}")
                print("-" * 30)
    
    def show_stats(b):
        with output:
            stats = face_system.get_database_stats()
            detector_stats = face_system.get_detector_stats()
            
            print("📊 Database Statistics:")
            print("=" * 50)
            print(f"Total faces: {stats['total_faces']}")
            print(f"Unique people: {stats['unique_people']}")
            print(f"Database size: {stats['database_size_mb']:.2f} MB")
            
            if stats['name_distribution']:
                print("\n👤 Face distribution:")
                for name, count in stats['name_distribution'].items():
                    print(f"  {name}: {count} face(s)")
            
            print("\n⚡ Detector Performance:")
            for detector, perf in detector_stats.items():
                print(f"  {detector}: {perf['detection_count']} detections, "
                      f"avg {perf['average_time_ms']:.2f}ms")
    
    # Bind events
    add_button.on_click(add_face)
    remove_button.on_click(remove_face)
    list_button.on_click(list_faces)
    stats_button.on_click(show_stats)
    
    # Layout
    input_box = widgets.VBox([name_input, description_input])
    button_box = widgets.HBox([add_button, remove_button, list_button, stats_button])
    
    return widgets.VBox([input_box, button_box, output])

# Display interface
database_interface = create_database_interface()
display(database_interface)

NameError: name 'FaceRecognitionSystem' is not defined

In [ ]:
# Test all detectors with a sample image or webcam frame
def test_detectors():
    print("🔍 Testing Face Detection Models...")
    print("=" * 50)
    
    # Capture a test frame
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ Could not open webcam")
        return
    
    ret, test_frame = cap.read()
    cap.release()
    
    if not ret:
        print("❌ Could not capture test frame")
        return
    
    # Test each detector
    results = {}
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    axes = axes.flatten()
    
    # Original frame
    axes[0].imshow(cv2.cvtColor(test_frame, cv2.COLOR_BGR2RGB))
    axes[0].set_title('Original Frame')
    axes[0].axis('off')
    
    detector_names = ['haar', 'dnn', 'mtcnn']
    for i, detector_name in enumerate(detector_names):
        print(f"Testing {detector_name.upper()} detector...")
        
        # Switch detector
        face_system.set_detector(detector_name)
        
        # Process frame
        start_time = time.time()
        frame_results = face_system.process_frame(test_frame)
        processing_time = time.time() - start_time
        
        # Draw results
        annotated_frame = face_system.draw_results(test_frame, frame_results)
        
        # Display
        axes[i + 1].imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
        axes[i + 1].set_title(f'{detector_name.upper()} - {len(frame_results["faces"])} faces - {processing_time*1000:.1f}ms')
        axes[i + 1].axis('off')
        
        results[detector_name] = {
            'faces_detected': len(frame_results['faces']),
            'processing_time_ms': processing_time * 1000,
            'detector_time_ms': frame_results.get('processing_time', 0) * 1000
        }
        
        print(f"  Faces detected: {results[detector_name]['faces_detected']}")
        print(f"  Processing time: {results[detector_name]['processing_time_ms']:.2f}ms")
    
    plt.tight_layout()
    plt.show()
    
    # Performance comparison
    print("\n📊 Performance Comparison:")
    print("=" * 50)
    df = pd.DataFrame(results).T
    df = df.round(2)
    print(df)
    
    return results

# Run detector tests
detector_test_results = test_detectors()

In [ ]:
# Create real-time processing interface
def create_realtime_interface():
    # Control widgets
    detector_dropdown = widgets.Dropdown(
        options=['haar', 'dnn', 'mtcnn'],
        value='haar',
        description='Detector:',
        style={'description_width': 'initial'}
    )
    
    tolerance_slider = widgets.FloatSlider(
        value=0.6,
        min=0.3,
        max=0.9,
        step=0.05,
        description='Tolerance:',
        style={'description_width': 'initial'}
    )
    
    confidence_slider = widgets.FloatSlider(
        value=0.4,
        min=0.1,
        max=0.9,
        step=0.05,
        description='Min Confidence:',
        style={'description_width': 'initial'}
    )
    
    threading_checkbox = widgets.Checkbox(
        value=False,
        description='Use Threading',
        style={'description_width': 'initial'}
    )
    
    anti_spoofing_checkbox = widgets.Checkbox(
        value=False,
        description='Anti-spoofing',
        style={'description_width': 'initial'}
    )
    
    start_button = widgets.Button(
        description='Start Recognition',
        button_style='success',
        icon='play'
    )
    
    stop_button = widgets.Button(
        description='Stop Recognition',
        button_style='danger',
        icon='stop'
    )
    
    screenshot_button = widgets.Button(
        description='Take Screenshot',
        button_style='info',
        icon='camera'
    )
    
    output = widgets.Output()
    
    # Global variables for the camera loop
    camera_active = False
    cap = None
    
    def update_settings(change=None):
        face_system.set_detector(detector_dropdown.value)
        face_system.recognition_tolerance = tolerance_slider.value
        face_system.min_confidence = confidence_slider.value
        face_system.enable_anti_spoofing = anti_spoofing_checkbox.value
    
    def start_camera(b):
        nonlocal camera_active, cap
        
        with output:
            if camera_active:
                print("⚠️ Camera is already running")
                return
            
            cap = cv2.VideoCapture(0)
            if not cap.isOpened():
                print("❌ Could not open webcam")
                return
            
            camera_active = True
            update_settings()
            
            if threading_checkbox.value:
                face_system.start_processing_thread()
            
            print("📹 Camera started. Press 'q' in the video window to stop.")
            
            while camera_active:
                ret, frame = cap.read()
                if not ret:
                    break
                
                # Process frame
                results = face_system.process_frame(frame, threading_checkbox.value)
                
                # Draw results
                annotated_frame = face_system.draw_results(frame, results)
                
                # Display
                cv2.imshow('Face Recognition System - Press Q to quit', annotated_frame)
                
                # Check for quit
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            # Cleanup
            camera_active = False
            if cap:
                cap.release()
            cv2.destroyAllWindows()
            face_system.stop_processing_thread()
            print("📹 Camera stopped")
    
    def stop_camera(b):
        nonlocal camera_active
        camera_active = False
    
    def take_screenshot(b):
        with output:
            if not camera_active or cap is None:
                print("❌ Camera is not running")
                return
            
            ret, frame = cap.read()
            if ret:
                timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
                filename = f"screenshot_{timestamp}.jpg"
                
                # Process and annotate frame
                results = face_system.process_frame(frame)
                annotated_frame = face_system.draw_results(frame, results)
                
                cv2.imwrite(filename, annotated_frame)
                print(f"📸 Screenshot saved as {filename}")
            else:
                print("❌ Could not capture frame")
    
    # Bind events
    detector_dropdown.observe(update_settings, names='value')
    tolerance_slider.observe(update_settings, names='value')
    confidence_slider.observe(update_settings, names='value')
    anti_spoofing_checkbox.observe(update_settings, names='value')
    
    start_button.on_click(start_camera)
    stop_button.on_click(stop_camera)
    screenshot_button.on_click(take_screenshot)
    
    # Layout
    controls_box1 = widgets.HBox([detector_dropdown, tolerance_slider])
    controls_box2 = widgets.HBox([confidence_slider, threading_checkbox, anti_spoofing_checkbox])
    button_box = widgets.HBox([start_button, stop_button, screenshot_button])
    
    return widgets.VBox([controls_box1, controls_box2, button_box, output])

# Display real-time interface
realtime_interface = create_realtime_interface()
display(realtime_interface)

In [ ]:
# Performance monitoring and analysis
def create_performance_monitor():
    monitor_output = widgets.Output()
    
    start_monitor_button = widgets.Button(
        description='Start Monitoring',
        button_style='info',
        icon='chart-line'
    )
    
    stop_monitor_button = widgets.Button(
        description='Stop Monitoring',
        button_style='warning',
        icon='stop'
    )
    
    monitoring_active = False
    
    def start_monitoring(b):
        nonlocal monitoring_active
        monitoring_active = True
        
        with monitor_output:
            print("📊 Starting performance monitoring...")
            
            # Performance data collection
            performance_data = {
                'timestamp': [],
                'cpu_percent': [],
                'memory_percent': [],
                'fps': [],
                'processing_time': []
            }
            
            # GPU data if available
            if cuda_manager.cuda_available:
                performance_data.update({
                    'gpu_utilization': [],
                    'gpu_memory_percent': []
                })
            
            start_time = time.time()
            
            while monitoring_active and (time.time() - start_time) < 30:  # Monitor for 30 seconds
                # Get system resources
                resources = cuda_manager.monitor_resources()
                
                performance_data['timestamp'].append(time.time() - start_time)
                performance_data['cpu_percent'].append(resources.get('cpu_percent', 0))
                performance_data['memory_percent'].append(resources.get('memory_percent', 0))
                performance_data['fps'].append(face_system.current_fps)
                
                # Get latest processing time
                if hasattr(face_system, '_last_processing_time'):
                    performance_data['processing_time'].append(face_system._last_processing_time * 1000)
                else:
                    performance_data['processing_time'].append(0)
                
                if cuda_manager.cuda_available:
                    performance_data['gpu_utilization'].append(resources.get('gpu_utilization', 0))
                    performance_data['gpu_memory_percent'].append(resources.get('gpu_memory_percent', 0))
                
                time.sleep(1)  # Sample every second
            
            monitoring_active = False
            
            # Plot results
            if len(performance_data['timestamp']) > 1:
                fig_rows = 3 if cuda_manager.cuda_available else 2
                fig, axes = plt.subplots(fig_rows, 2, figsize=(15, 4 * fig_rows))
                
                # CPU Usage
                axes[0, 0].plot(performance_data['timestamp'], performance_data['cpu_percent'])
                axes[0, 0].set_title('CPU Usage (%)')
                axes[0, 0].set_ylabel('Usage %')
                axes[0, 0].grid(True)
                
                # Memory Usage
                axes[0, 1].plot(performance_data['timestamp'], performance_data['memory_percent'])
                axes[0, 1].set_title('Memory Usage (%)')
                axes[0, 1].set_ylabel('Usage %')
                axes[0, 1].grid(True)
                
                # FPS
                axes[1, 0].plot(performance_data['timestamp'], performance_data['fps'])
                axes[1, 0].set_title('Frames Per Second')
                axes[1, 0].set_ylabel('FPS')
                axes[1, 0].grid(True)
                
                # Processing Time
                axes[1, 1].plot(performance_data['timestamp'], performance_data['processing_time'])
                axes[1, 1].set_title('Processing Time (ms)')
                axes[1, 1].set_ylabel('Time (ms)')
                axes[1, 1].grid(True)
                
                if cuda_manager.cuda_available:
                    # GPU Utilization
                    axes[2, 0].plot(performance_data['timestamp'], performance_data['gpu_utilization'])
                    axes[2, 0].set_title('GPU Utilization (%)')
                    axes[2, 0].set_ylabel('Usage %')
                    axes[2, 0].grid(True)
                    
                    # GPU Memory
                    axes[2, 1].plot(performance_data['timestamp'], performance_data['gpu_memory_percent'])
                    axes[2, 1].set_title('GPU Memory (%)')
                    axes[2, 1].set_ylabel('Usage %')
                    axes[2, 1].grid(True)
                
                for ax in axes.flat:
                    ax.set_xlabel('Time (seconds)')
                
                plt.tight_layout()
                plt.show()
                
                # Summary statistics
                print("\n📈 Performance Summary:")
                print("=" * 50)
                print(f"Average FPS: {np.mean(performance_data['fps']):.2f}")
                print(f"Average Processing Time: {np.mean(performance_data['processing_time']):.2f}ms")
                print(f"Average CPU Usage: {np.mean(performance_data['cpu_percent']):.1f}%")
                print(f"Average Memory Usage: {np.mean(performance_data['memory_percent']):.1f}%")
                
                if cuda_manager.cuda_available:
                    print(f"Average GPU Utilization: {np.mean(performance_data['gpu_utilization']):.1f}%")
                    print(f"Average GPU Memory: {np.mean(performance_data['gpu_memory_percent']):.1f}%")
            
            print("✅ Monitoring completed")
    
    def stop_monitoring(b):
        nonlocal monitoring_active
        monitoring_active = False
        with monitor_output:
            print("⏹️ Monitoring stopped")
    
    start_monitor_button.on_click(start_monitoring)
    stop_monitor_button.on_click(stop_monitoring)
    
    button_box = widgets.HBox([start_monitor_button, stop_monitor_button])
    return widgets.VBox([button_box, monitor_output])

# Display performance monitor
performance_monitor = create_performance_monitor()
display(performance_monitor)

In [ ]:
# Testing and validation suite
def run_comprehensive_tests():
    print("🧪 Running Comprehensive Test Suite")
    print("=" * 60)
    
    # Test 1: Database operations
    print("\n1️⃣ Testing Database Operations:")
    print("-" * 30)
    
    initial_count = face_system.database.get_face_count()
    print(f"Initial face count: {initial_count}")
    
    # Test adding sample face (if we have sample images)
    sample_dir = "sample_images"
    if os.path.exists(sample_dir):
        sample_files = [f for f in os.listdir(sample_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if sample_files:
            sample_file = os.path.join(sample_dir, sample_files[0])
            test_name = "TestPerson"
            
            success = face_system.database.add_face_from_image(sample_file, test_name, "Test face")
            if success:
                print(f"✅ Successfully added test face")
                # Remove it
                face_system.database.remove_face(test_name)
                print(f"✅ Successfully removed test face")
            else:
                print(f"❌ Failed to add test face")
    
    # Test 2: Detector performance
    print("\n2️⃣ Testing Detector Performance:")
    print("-" * 30)
    
    # Create test image
    test_image = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
    
    for detector_name in ['haar', 'dnn', 'mtcnn']:
        face_system.set_detector(detector_name)
        
        # Time multiple detections
        times = []
        for _ in range(10):
            start = time.time()
            results = face_system.process_frame(test_image)
            times.append((time.time() - start) * 1000)
        
        avg_time = np.mean(times)
        std_time = np.std(times)
        
        print(f"{detector_name.upper()}: {avg_time:.2f}±{std_time:.2f}ms")
    
    # Test 3: Memory usage
    print("\n3️⃣ Testing Memory Usage:")
    print("-" * 30)
    
    import psutil
    process = psutil.Process()
    
    # Baseline memory
    baseline_memory = process.memory_info().rss / 1024 / 1024  # MB
    print(f"Baseline memory: {baseline_memory:.1f} MB")
    
    # Process many frames
    for i in range(100):
        test_frame = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
        face_system.process_frame(test_frame)
    
    # Check memory after processing
    after_memory = process.memory_info().rss / 1024 / 1024  # MB
    memory_increase = after_memory - baseline_memory
    
    print(f"Memory after processing: {after_memory:.1f} MB")
    print(f"Memory increase: {memory_increase:.1f} MB")
    
    if memory_increase < 50:  # Less than 50MB increase is acceptable
        print("✅ Memory usage within acceptable limits")
    else:
        print("⚠️ High memory usage detected")
    
    # Test 4: Threading performance
    print("\n4️⃣ Testing Threading Performance:")
    print("-" * 30)
    
    # Test without threading
    start_time = time.time()
    for i in range(50):
        test_frame = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
        face_system.process_frame(test_frame, use_threading=False)
    sync_time = time.time() - start_time
    
    # Test with threading
    face_system.start_processing_thread()
    start_time = time.time()
    for i in range(50):
        test_frame = np.random.randint(0, 255, (480, 640, 3), dtype=np.uint8)
        face_system.process_frame(test_frame, use_threading=True)
    async_time = time.time() - start_time
    face_system.stop_processing_thread()
    
    print(f"Synchronous processing: {sync_time:.2f}s")
    print(f"Asynchronous processing: {async_time:.2f}s")
    
    if async_time < sync_time:
        speedup = sync_time / async_time
        print(f"✅ Threading provides {speedup:.2f}x speedup")
    else:
        print("⚠️ Threading overhead detected")
    
    # Test 5: CUDA functionality (if available)
    if cuda_manager.cuda_available:
        print("\n5️⃣ Testing CUDA Functionality:")
        print("-" * 30)
        
        try:
            # Test GPU memory operations
            test_tensor = torch.randn(1000, 1000).cuda()
            gpu_computation = torch.mm(test_tensor, test_tensor)
            result = gpu_computation.cpu()
            print("✅ CUDA tensor operations working")
            
            # Clean up GPU memory
            del test_tensor, gpu_computation
            torch.cuda.empty_cache()
            print("✅ GPU memory cleanup successful")
            
        except Exception as e:
            print(f"❌ CUDA test failed: {e}")
    
    print("\n" + "=" * 60)
    print("🎉 Test Suite Completed!")

# Run the comprehensive tests
run_comprehensive_tests()

In [ ]:
# Advanced features demonstration
def demonstrate_advanced_features():
    print("🚀 Advanced Features Demonstration")
    print("=" * 50)
    
    # Feature 1: Batch face processing
    print("\n1️⃣ Batch Face Processing:")
    print("-" * 30)
    
    # Create multiple test frames
    batch_frames = []
    for i in range(5):
        frame = np.random.randint(0, 255, (240, 320, 3), dtype=np.uint8)
        # Add some noise patterns to simulate faces
        cv2.rectangle(frame, (50, 50), (150, 150), (255, 255, 255), -1)
        cv2.rectangle(frame, (70, 70), (80, 90), (0, 0, 0), -1)  # Left eye
        cv2.rectangle(frame, (120, 70), (130, 90), (0, 0, 0), -1)  # Right eye
        cv2.rectangle(frame, (90, 110), (110, 130), (0, 0, 0), -1)  # Nose
        batch_frames.append(frame)
    
    start_time = time.time()
    batch_results = []
    for frame in batch_frames:
        result = face_system.process_frame(frame)
        batch_results.append(result)
    
    batch_time = time.time() - start_time
    print(f"Processed {len(batch_frames)} frames in {batch_time:.2f}s")
    print(f"Average time per frame: {(batch_time/len(batch_frames))*1000:.2f}ms")
    
    # Feature 2: Recognition confidence analysis
    print("\n2️⃣ Recognition Confidence Analysis:")
    print("-" * 30)
    
    if face_system.database.get_face_count() > 0:
        confidence_scores = []
        for result in batch_results:
            for face in result.get('faces', []):
                if face['name'] != 'Unknown':
                    confidence_scores.append(face['confidence'])
        
        if confidence_scores:
            print(f"Recognition attempts: {len(confidence_scores)}")
            print(f"Average confidence: {np.mean(confidence_scores):.3f}")
            print(f"Confidence std dev: {np.std(confidence_scores):.3f}")
            
            # Plot confidence distribution
            plt.figure(figsize=(10, 6))
            plt.hist(confidence_scores, bins=20, alpha=0.7, edgecolor='black')
            plt.axvline(np.mean(confidence_scores), color='red', linestyle='--', 
                       label=f'Mean: {np.mean(confidence_scores):.3f}')
            plt.xlabel('Confidence Score')
            plt.ylabel('Frequency')
            plt.title('Recognition Confidence Distribution')
            plt.legend()
            plt.grid(True, alpha=0.3)
            plt.show()
        else:
            print("No recognized faces for confidence analysis")
    else:
        print("No faces in database for analysis")
    
    # Feature 3: System resource monitoring
    print("\n3️⃣ Real-time Resource Monitoring:")
    print("-" * 30)
    
    monitoring_data = cuda_manager.monitor_resources()
    
    print("Current System Status:")
    for key, value in monitoring_data.items():
        if isinstance(value, float):
            print(f"  {key}: {value:.2f}")
        else:
            print(f"  {key}: {value}")
    
    # Feature 4: Export/Import demonstration
    print("\n4️⃣ Database Export/Import:")
    print("-" * 30)
    
    export_path = "database_backup"
    success = face_system.database.export_database(export_path)
    if success:
        print(f"✅ Database exported to {export_path}")
        
        # Calculate backup size
        backup_size = sum(
            os.path.getsize(os.path.join(dirpath, filename))
            for dirpath, dirnames, filenames in os.walk(export_path)
            for filename in filenames
        ) / (1024 * 1024)  # MB
        
        print(f"Backup size: {backup_size:.2f} MB")
    else:
        print("❌ Database export failed")
    
    # Feature 5: Performance optimization
    print("\n5️⃣ Performance Optimization:")
    print("-" * 30)
    
    # Memory optimization
    cuda_manager.optimize_memory()
    print("✅ Memory optimization completed")
    
    # Get detector statistics
    detector_stats = face_system.get_detector_stats()
    
    print("\nDetector Performance Summary:")
    for detector, stats in detector_stats.items():
        if stats['detection_count'] > 0:
            print(f"  {detector.upper()}:")
            print(f"    Detections: {stats['detection_count']}")
            print(f"    Avg time: {stats['average_time_ms']:.2f}ms")
            print(f"    Total time: {stats['total_time']:.2f}s")

# Run advanced features demo
demonstrate_advanced_features()

In [ ]:
# Final cleanup and instructions
def cleanup_system():
    print("🧹 Cleaning Up System Resources")
    print("=" * 50)
    
    # Stop any running processes
    face_system.cleanup()
    
    # Clear GPU memory
    cuda_manager.optimize_memory()
    
    # Close any open CV windows
    cv2.destroyAllWindows()
    
    print("✅ Cleanup completed")
    
    print("\n📋 Usage Instructions:")
    print("=" * 50)
    print("1. Use the Database Management interface to add/remove faces")
    print("2. Configure detection settings in the Real-time interface")
    print("3. Start camera recognition with the 'Start Recognition' button")
    print("4. Press 'q' in the video window to stop recognition")
    print("5. Use performance monitoring to optimize settings")
    print("6. Take screenshots during recognition for analysis")
    
    print("\n⚙️ System Configuration:")
    print("=" * 30)
    print(f"CUDA Available: {cuda_manager.cuda_available}")
    print(f"OpenCV CUDA: {cuda_manager.opencv_cuda_available}")
    print(f"Face Database: {face_system.database.get_face_count()} faces")
    print(f"Current Detector: {face_system.current_detector}")
    print(f"Recognition Tolerance: {face_system.recognition_tolerance}")
    
    print("\n🔧 Troubleshooting:")
    print("=" * 30)
    print("- If camera doesn't open: Check camera permissions and connections")
    print("- If CUDA errors occur: Verify CUDA installation and GPU compatibility")
    print("- If recognition is poor: Adjust tolerance and add more face samples")
    print("- If performance is slow: Try different detectors or enable threading")
    
    print("\n🎯 Performance Targets Achieved:")
    print("=" * 30)
    current_fps = getattr(face_system, 'current_fps', 0)
    if current_fps > 20:
        print(f"✅ FPS: {current_fps:.1f} (Target: >20)")
    else:
        print(f"⚠️ FPS: {current_fps:.1f} (Target: >20)")
    
    # Get recent processing times from detector stats
    detector_stats = face_system.get_detector_stats()
    avg_times = [stats['average_time_ms'] for stats in detector_stats.values() 
                if stats['detection_count'] > 0]
    
    if avg_times:
        min_time = min(avg_times)
        if min_time < 100:
            print(f"✅ Latency: {min_time:.1f}ms (Target: <100ms)")
        else:
            print(f"⚠️ Latency: {min_time:.1f}ms (Target: <100ms)")
    
    database_count = face_system.database.get_face_count()
    print(f"✅ Database: {database_count} faces stored")
    
    print("\n🚀 Ready for Production Use!")

# Show final system status
print("🎉 Face Recognition System Setup Complete!")
print("=" * 60)

# Display final statistics
stats = face_system.get_database_stats()
detector_stats = face_system.get_detector_stats()

print(f"📊 Final System Statistics:")
print(f"  Database: {stats['total_faces']} faces, {stats['unique_people']} people")
print(f"  System: {cuda_manager.device} processing")
print(f"  Memory: {cuda_manager.get_system_info()['memory_available']} GB available")

if cuda_manager.cuda_available:
    gpu_info = cuda_manager.get_system_info()
    print(f"  GPU: {gpu_info['gpu_name']}")
    print(f"  GPU Memory: {gpu_info['gpu_memory_total']} GB total")

# Run cleanup
cleanup_system()